In [1]:
!git clone https://github.com/starbucksdolcelatte/ShowMeTheColor

fatal: destination path 'ShowMeTheColor' already exists and is not an empty directory.


In [1]:
from tensorflow.keras.applications.mobilenet import  MobileNet
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense,Flatten,Input
import os

In [2]:
size=(224,224,3)
size

(224, 224, 3)

In [3]:
image=ImageDataGenerator(rescale=1/255)

In [4]:
train=image.flow_from_directory(os.path.join('img','train'),target_size=size[:2],batch_size=32,shuffle=True)
test=image.flow_from_directory(os.path.join('img','test'),target_size=size[:2],batch_size=32,shuffle=True)

Found 213 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [5]:
mobilenet=MobileNet(include_top=False,input_shape=size)
mobilenet.summary()

for layer in mobilenet.layers:
    layer.trainable = False

Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [ ]:
model=Sequential((Input(shape=size),mobilenet,Flatten(),Dense(64,activation="relu"),Dense(train,activation="softmax")))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,440,452 (24.57 MB)

 Trainable params: 3,211,588 (12.25 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [9]:
model.fit(train,epochs=500)

Epoch 1/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.4817 - loss: 1.0100
Epoch 2/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.4756 - loss: 0.9860
Epoch 3/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.4787 - loss: 1.0040
Epoch 4/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.4853 - loss: 1.0385
Epoch 5/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.4900 - loss: 0.9679
Epoch 6/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step - accuracy: 0.4609 - loss: 1.0597
Epoch 7/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.4237 - loss: 1.0169
Epoch 8/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.4636 - loss: 0.9916
Epoch 9/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - accuracy: 0.4786 - loss: 0.9776
Epoch 10/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.5289 - loss: 0.8960
Epoch 11/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.4842 - loss: 0.9090
Epoch 12/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy:

In [10]:
model.save('model.keras')